In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15469827
paper_name = 'begley_samson_2004' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [17]:
original_data = pd.read_excel('raw_data/Begley2003.xlsx', sheet_name='Sheet1')

In [18]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4852 x 19


In [19]:
original_data.head()

,ORF,Gene,YPD Name/Description of Protein,plate,position,Sensitivity scores (G-J),MMS score,tOH score,4NQO score,UV score,Sensitvity / restant (L-S),MMS sensitive,MMS resistant,tOH sensitive,tOH resistant,4NQO sensitive,4NQO resistant,UV sensitive,UV resistant
0,YAL002W,VPS8,Protein involved in vacuolar sorting,1,3E,Score = 0 Is not sensitive,4,9,0,0,Sensitve / resistant = 1,1,0,1,0,0,0,0,0
1,YAL004W,NaN,"Protein of unknown function, probable non-codi...",1,1E,Score = 2 Is minimum sensitive,0,0,0,0,not sensitve = 0,0,0,0,0,0,0,0,0
2,YAL005C,SSA1,Cytoplasmic chaperone and heat shock protein o...,1,2E,Score = 30 is maximum sensitive,0,0,0,0,NaN,0,0,0,0,0,0,0,0
3,YAL007C,ERP2,Protein with similarity to COPII-coated vesicl...,1,12D,NaN,0,0,0,0,NaN,0,0,0,0,0,0,0,0
4,YAL008W,FUN14,Protein of unknown function,1,11D,NaN,0,0,0,0,NaN,0,0,0,0,0,0,0,0


In [20]:
original_data['orf'] = original_data['ORF'].astype(str)

In [21]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [22]:
typo_fixes = {'YAR002AW':'YAR002W-A','YFL033AC':'YFL033C-A','YKLO72W':'YKL072W',
              'YOLO57W':'YOL057W','YOLO62C':'YOL062C','YJL206-A':'YJL205C-A','YLR287-A':'YLR287C-A'}
original_data['orf'] = original_data['orf'].apply(lambda x: typo_fixes[x] if x in typo_fixes.keys() else x)

In [23]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [24]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Gene, YPD Name/Description of Protein, plate , position, Sensitivity scores (G-J), MMS score, tOH score, 4NQO score, UV score, Sensitvity / restant (L-S), MMS sensitive, MMS resistant, tOH sensitive, tOH resistant, 4NQO sensitive, 4NQO resistant, UV sensitive, UV resistant, orf]
Index: []


In [25]:
original_data.set_index('orf', inplace=True)

In [26]:
# The sensitivity scores are such that the higher the number, the more sensitive the mutant.
original_data = -original_data[['MMS score','tOH score','4NQO score','UV score']].apply(pd.to_numeric, axis=1, errors='coerce')

In [27]:
original_data = original_data.groupby(original_data.index).mean()

In [28]:
original_data.shape

(4759, 4)

# Prepare the final dataset

In [29]:
data = original_data.copy()

In [30]:
dataset_ids = [111, 543, 544, 545]
datasets = datasets.reindex(index=dataset_ids)

In [31]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [32]:
data.head()

dataset_id,111,543,544,545
data_type,value,value,value,value
orf,,,,
YAL002W,-4.0,-9.0,0.0,0.0
YAL004W,0.0,0.0,0.0,0.0
YAL005C,0.0,0.0,0.0,0.0
YAL007C,0.0,0.0,0.0,0.0
YAL008W,0.0,0.0,0.0,0.0


## Subset to the genes currently in SGD

In [33]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 28


In [34]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,111,543,544,545
,data_type,value,value,value,value
gene_id,orf,,,,
2,YAL002W,-4.0,-9.0,0.0,0.0
1863,YAL004W,0.0,0.0,0.0,0.0
4,YAL005C,0.0,0.0,0.0,0.0
5,YAL007C,0.0,0.0,0.0,0.0
6,YAL008W,0.0,0.0,0.0,0.0


# Normalize

In [35]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [36]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [37]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,111,543,544,545,111,543,544,545
,data_type,value,value,value,value,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,
2,YAL002W,-4.0,-9.0,0.0,0.0,-0.762858,-6.463717,0.0,0.0
1863,YAL004W,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,YAL005C,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
5,YAL007C,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
6,YAL008W,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


# Print out

In [38]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [39]:
from IO.save_data_to_db3 import *

In [40]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/4 [00:00<?, ?it/s]

Deleting all datasets for PMID 15469827...
Inserting the new data...


100%|██████████| 4/4 [00:28<00:00,  7.06s/it]

Updating the data_modified_on field...
